In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import os

In [14]:
def list_files_in_directory(directory):
    # List all files in the given directory
    filenames = []
    for filename in os.listdir(directory):
        # Construct the full file path
        file_path = os.path.join(directory, filename)
        # Check if it is a file (not a directory)
        if os.path.isfile(file_path):
            filenames.append(os.path.join(directory, filename))
    return filenames


In [17]:
"""directory_path = './Potato___healthy'
files = list_files_in_directory(directory_path)
print(files)"""
healthy_files[0]

'./Potato___healthy/61b8a692-91d8-40f5-a4a4-4d910875a638___RS_HL 1966.JPG'

In [18]:
healthy_files = list_files_in_directory("./Potato___healthy")
early_blights = list_files_in_directory("./Potato___Early_blight")
late_blights = list_files_in_directory("./Potato___Late_blight")


In [19]:
# Custom Dataset
class PlantDataset(Dataset):
    def __init__(self, healthy_files, early_blights_files, late_blights_files, transform=None):
        self.filenames = healthy_files + early_blights_files + late_blights_files
        self.labels = [0] * len(healthy_files) + [1] * len(early_blights_files) + [2] * len(late_blights_files) # 0 for cat, 1 for dog
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_path = self.filenames[idx]
        image = Image.open(img_path)
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

In [20]:
# Create dataset and dataloader
dataset = PlantDataset(healthy_files, early_blights, late_blights, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [24]:
# Define the model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(32 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 3)  # 2 classes: cat and dog
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 32 * 32)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [25]:
# Initialize the model, loss function and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [26]:
num_epochs = 10

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in dataloader:
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')

# Save the model
torch.save(model.state_dict(), 'plant_disease.pth')

Epoch [1/10], Loss: 0.4980
Epoch [2/10], Loss: 0.1879
Epoch [3/10], Loss: 0.1162
Epoch [4/10], Loss: 0.1042
Epoch [5/10], Loss: 0.0677
Epoch [6/10], Loss: 0.0632
Epoch [7/10], Loss: 0.0751
Epoch [8/10], Loss: 0.0576
Epoch [9/10], Loss: 0.0259
Epoch [10/10], Loss: 0.0121
